In [ ]:
!pip install --upgrade pip setuptools wheel
!pip install "pydantic<2.0" typing-extensions==4.7.1
!pip install chromadb sentence-transformers spacy pandas scikit-learn
!python -m spacy download en_core_web_sm


  Using cached pydantic-1.10.22-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (154 kB)
  Using cached typing_extensions-4.7.1-py3-none-any.whl.metadata (3.1 kB)
Using cached typing_extensions-4.7.1-py3-none-any.whl (33 kB)
Using cached pydantic-1.10.22-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.15.0
    Uninstalling typing_extensions-4.15.0:
      Successfully uninstalled typing_extensions-4.15.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.11.7
    Uninstalling pydantic-2.11.7:
      Successfully uninstalled pydantic-2.11.7
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pydantic]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mcp 1.13.1 requires pydantic<3.0.0,>=2.11.0, but you have pyd

In [ ]:
import os  # for path operations
import math  # for ceiling when batching
import pandas as pd  # for dataframe operations
import chromadb  # the vector DB
from chromadb.config import Settings  # to configure persistence
from sentence_transformers import SentenceTransformer  # huggingface model for embeddings
import spacy  # for lemmatization and basic NLP cleaning
import re  # regular expressions for light cleaning
from typing import List, Dict, Tuple

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
CSV_PATH = "/content/drive/MyDrive/imdb_top_1000.csv"
MODEL_NAME = "all-MiniLM-L6-v2"
EMBEDDING_BATCH_SIZE = 64
CHUNK_SIZE_WORDS = 120
CHROMA_PERSIST_DIR = "./chroma_movie_db"
TOP_K = 5

In [ ]:

nlp = spacy.load("en_core_web_sm", disable=["ner"])

def simple_clean_text(text: str) -> str:
    """Light cleaning: normalize whitespace and remove weird characters."""
    if not isinstance(text, str):
        return ""
    text = text.strip()
    text = re.sub(r"[\r\n]+", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text

def lemmatize_and_filter(text: str, remove_stopwords: bool = True) -> str:
    """Use spaCy to lemmatize text and optionally remove stopwords.

    Returns a cleaned string suitable for embedding.
    """
    text = simple_clean_text(text)
    if text == "":
        return ""
    doc = nlp(text)
    lemmas = []
    for token in doc:

        if token.is_punct or token.is_space:
            continue
        # optionally skip stopwords
        if remove_stopwords and token.is_stop:
            continue
        # take lemma and lower it
        lemmas.append(token.lemma_.lower())
    return " ".join(lemmas)

def chunk_text_by_words(text: str, max_words: int = CHUNK_SIZE_WORDS) -> List[str]:
    """Split text into chunks of up to max_words words.

    This helps produce embeddings for long descriptions without losing context.
    """
    words = text.split()
    if len(words) <= max_words:
        return [text]
    chunks = []
    for i in range(0, len(words), max_words):
        chunk = " ".join(words[i : i + max_words])
        chunks.append(chunk)
    return chunks

In [ ]:
# ---------- 5) Load CSV into a pandas DataFrame and basic cleaning ----------
def load_and_prepare_dataframe(csv_path: str = CSV_PATH) -> pd.DataFrame:
    """Load CSV, pick useful columns, drop missing rows, and create a 'text' column.

    Expects the CSV to have at least columns: Series_Title and Overview.
    """
    # read CSV into pandas
    df = pd.read_csv(csv_path)  # read file from given path

    # Explicitly set title and description columns based on the CSV header
    title_col = "Series_Title"
    desc_col = "Overview"

    # Check if the required columns exist
    if title_col not in df.columns:
        raise ValueError(f"CSV must contain a '{title_col}' column.")
    if desc_col not in df.columns:
         raise ValueError(f"CSV must contain a '{desc_col}' column.")


    # keep only the columns we need and rename them to standard names
    df = df[[title_col, desc_col]].copy()
    df.columns = ["title", "description"]

    # drop rows where title and description are both missing
    df.dropna(subset=["title", "description"], how="all", inplace=True)

    # fill missing descriptions with empty string
    df["description"] = df["description"].fillna("")

    # create a combined text column we will embed (title + description)
    df["combined_raw"] = df["title"].astype(str) + ". " + df["description"].astype(str)

    return df.reset_index(drop=True)

In [ ]:

# Build documents to index: cleaning, lemmatization, chunking
def build_documents(df: pd.DataFrame) -> List[Dict]:
    """From the dataframe, build a list of dicts representing documents/chunks.

    Each dict will have: id, movie_idx (row index), title, chunk_text, original_text.
    """
    docs = []  # list to hold all small documents/chunks
    for idx, row in df.iterrows():
        title = str(row["title"])  # movie title
        full_text = str(row["combined_raw"])  # title + description combined

        # light-clean the raw combined text
        cleaned = simple_clean_text(full_text)

        # chunk the cleaned text into smaller pieces
        chunks = chunk_text_by_words(cleaned, max_words=CHUNK_SIZE_WORDS)

        # for each chunk, lemmatize and create a document entry
        for chunk_i, chunk in enumerate(chunks):
            # lemmatize the chunk (removes stopwords by default)
            lemma = lemmatize_and_filter(chunk, remove_stopwords=True)

            # create a unique id for this chunk: f"rowidx_chunkidx"
            doc_id = f"{idx}_{chunk_i}"

            docs.append(
                {
                    "id": doc_id,  # unique id for Chroma
                    "movie_idx": int(idx),  # original dataframe index
                    "title": title,  # movie title (metadata)
                    "chunk_text": lemma,  # cleaned + lemmatized text to embed
                    "original_text": chunk,  # original chunk (for display)
                }
            )
    return docs

In [ ]:

embed_model = None

def load_embedding_model(model_name: str = MODEL_NAME):
    """Load the sentence-transformers model (cached by the library).

    We keep the model in a global variable to avoid reloading.
    """
    global embed_model
    if embed_model is None:
        embed_model = SentenceTransformer(model_name)
    return embed_model

def embed_texts(texts: List[str], batch_size: int = EMBEDDING_BATCH_SIZE) -> List[List[float]]:
    """Embed a list of texts in batches and return list of vectors.

    This uses the sentence-transformers encode(...) function directly (no wrapper).
    """
    model = load_embedding_model()
    embeddings = []  # will collect embeddings
    n = len(texts)
    # process in batches
    for i in range(0, n, batch_size):
        batch = texts[i : i + batch_size]
        batch_emb = model.encode(batch, show_progress_bar=False, convert_to_numpy=True)
        embeddings.extend(batch_emb.tolist())
    return embeddings

In [ ]:
# Save to ChromaDB
def create_chroma_collection(persist_dir: str = CHROMA_PERSIST_DIR, collection_name: str = "movies"):
    """Create or open a ChromaDB collection with persistence configured.

    We do not supply an embedding_function to Chroma; we will pass embeddings explicitly.
    """
    # configure chroma to persist to a folder so it survives restarts
    client = chromadb.Client(Settings(persist_directory=persist_dir))

    # create or get collection
    try:
        collection = client.create_collection(name=collection_name)
    except Exception:
        collection = client.get_collection(name=collection_name)
    return client, collection

def index_documents_to_chroma(docs: List[Dict], collection, batch_size: int = EMBEDDING_BATCH_SIZE):
    """Index the list of docs into Chroma by computing embeddings and adding in batches.

    Each add uses explicit embeddings argument to avoid any Chroma embedding wrappers.
    """
    # prepare lists
    ids = [d["id"] for d in docs]
    metadatas = [
        {"movie_idx": d["movie_idx"], "title": d["title"], "original_text": d["original_text"]}
        for d in docs
    ]
    documents = [d["chunk_text"] for d in docs]  # texts we will embed

    # embed documents in batches to avoid memory blowups
    for i in range(0, len(documents), batch_size):
        batch_texts = documents[i : i + batch_size]
        batch_ids = ids[i : i + batch_size]
        batch_metas = metadatas[i : i + batch_size]

        # compute embeddings for this batch
        batch_emb = embed_texts(batch_texts, batch_size=batch_size)

        # add to chroma with explicit embeddings
        collection.add(ids=batch_ids, documents=batch_texts, metadatas=batch_metas, embeddings=batch_emb)

In [ ]:
def query_movies(query: str, collection, top_k: int = TOP_K) -> List[Dict[str, str]]:
    """Return top_k similar movies for a textual query.

    Returns a list of dicts with title and matched snippet (original_text) and score.
    """
    # clean + lemmatize query the same way we processed docs
    q_clean = lemmatize_and_filter(query, remove_stopwords=True)

    # embed the cleaned query
    q_emb = embed_texts([q_clean], batch_size=1)[0]

    # ask chroma for similar vectors via embeddings
    res = collection.query(query_embeddings=[q_emb], n_results=top_k, include=['metadatas', 'distances'])

    # Debugging: Print the structure of the results returned by ChromaDB
    print("ChromaDB query result structure:")
    print(res)


    # res contains 'ids', 'metadatas', 'distances' (each is a list per query)
    results = []
    if len(res.get('metadatas', [])) > 0:
        for meta, dist in zip(res['metadatas'][0], res['distances'][0]):
            # Debugging: Print each metadata dictionary
            print("Metadata for a result:")
            print(meta)

            results.append(
                {
                    'title': meta.get('title', ''),
                    'matched_snippet': meta.get('original_text', ''),
                    'distance': float(dist),
                    'movie_idx': meta.get('movie_idx', None) # Safely get movie_idx
                }
            )
    return results

In [ ]:
# ---------- 10) Putting it together: high-level run function ----------
def build_and_index(csv_path: str = CSV_PATH, persist_dir: str = CHROMA_PERSIST_DIR):
    """Load CSV, build docs, create chroma, and index all documents.

    Returns (client, collection, dataframe) for user interaction.
    """
    # 1. load dataframe
    df = load_and_prepare_dataframe(csv_path)

    # 2. build documents (clean + chunk + lemmatize)
    docs = build_documents(df)

    # 3. create chroma client and collection (persistent)
    client, collection = create_chroma_collection(persist_dir=persist_dir)

    # 4. index documents into chroma
    index_documents_to_chroma(docs, collection)

    # 5. persist chroma to disk (so it can be reopened later)
    # client.persist() # This line is no longer needed in newer ChromaDB versions

    return client, collection, df

In [ ]:

if __name__ == "__main__":
    # main runs when the script is called directly

    # 1) build the DB and index all movies from the CSV
    print("Loading CSV, cleaning text, building embeddings, and indexing into ChromaDB...")
    client, collection, df = build_and_index()
    print("Indexing complete. You can now run queries.")

Loading CSV, cleaning text, building embeddings, and indexing into ChromaDB...
Indexing complete. You can now run queries.


In [ ]:
# 2) query example (change the query text to experiment)
query_text = "space adventure with a strong female lead"
print(f"Querying for: {query_text}")
hits = query_movies(query_text, collection, top_k=5)

# 3) show results with Gemini explanation
print("\n--- Recommended Movies ---")
for i, h in enumerate(hits):
    print(f"\nResult {i+1}: title={h['title']}")

    # Get the full description from the original dataframe using the movie_idx
    original_movie_idx = h['movie_idx']
    full_description = df.loc[original_movie_idx, 'description']

    # Use Gemini to generate an explanation
    try:
        prompt = f"""
You are a witty movie guide recommending films.
Explain why the movie '{h['title']}' with the description '{full_description}'
is an exciting match for the query '{query_text}'.

- Do NOT repeat the same sentence structure each time.
- Make the explanation engaging and a bit creative (like a movie critic or storyteller).
- Highlight what makes the movie unique or appealing to someone with that query.
- Keep the explanation short (2–3 sentences).
"""
        explanation_response = gemini_model.generate_content(prompt)
        explanation = explanation_response.text
        print(f"Explanation: {explanation}")

    except Exception as e:
        print(f"Could not generate explanation: {e}")
        print(f"Matched snippet: {h['matched_snippet'][:300]}...") # Fallback to showing snippet

    print(f"Distance: {h['distance']:.4f}")

print("\n--- End of Recommendations ---")

Querying for: space adventure with a strong female lead
ChromaDB query result structure:
{'ids': [['575_0', '582_0', '223_0', '357_0', '796_0']], 'embeddings': None, 'documents': None, 'uris': None, 'included': ['metadatas', 'distances'], 'data': None, 'metadatas': [[{'title': 'Hidden Figures', 'movie_idx': 575, 'original_text': 'Hidden Figures. The story of a team of female African-American mathematicians who served a vital role in NASA during the early years of the U.S. space program.'}, {'movie_idx': 582, 'title': 'Rogue One', 'original_text': 'Rogue One. The daughter of an Imperial scientist joins the Rebel Alliance in a risky move to steal the plans for the Death Star.'}, {'original_text': 'Mad Max: Fury Road. In a post-apocalyptic wasteland, a woman rebels against a tyrannical ruler in search for her homeland with the aid of a group of female prisoners, a psychotic worshiper, and a drifter named Max.', 'movie_idx': 223, 'title': 'Mad Max: Fury Road'}, {'movie_idx': 357, 'title': 

In [ ]:
print(CSV_PATH)

In [ ]:
!ls -l "$CSV_PATH"

In [ ]:
display(df.head())

,title,description,combined_raw
0,The Shawshank Redemption,Two imprisoned men bond over a number of years...,The Shawshank Redemption. Two imprisoned men b...
1,The Godfather,An organized crime dynasty's aging patriarch t...,The Godfather. An organized crime dynasty's ag...
2,The Dark Knight,When the menace known as the Joker wreaks havo...,The Dark Knight. When the menace known as the ...
3,The Godfather: Part II,The early life and career of Vito Corleone in ...,The Godfather: Part II. The early life and car...
4,12 Angry Men,A jury holdout attempts to prevent a miscarria...,12 Angry Men. A jury holdout attempts to preve...


In [ ]:
csv_path = "/content/drive/MyDrive/imdb_top_1000.csv" # Make sure this path is correct
num_lines_to_show = 5

try:
    with open(csv_path, 'r', encoding='utf-8') as f:
        for i in range(num_lines_to_show):
            line = f.readline()
            if not line:
                break
            print(line.strip())
except FileNotFoundError:
    print(f"Error: The file was not found at {csv_path}")
except Exception as e:
    print(f"An error occurred: {e}")

Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
"https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UX67_CR0,0,67,98_AL_.jpg",The Shawshank Redemption,1994,A,142 min,Drama,9.3,"Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.",80,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
"https://m.media-amazon.com/images/M/MV5BM2MyNjYxNmUtYTAwNi00MTYxLWJmNWYtYzZlODY3ZTk3OTFlXkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_UY98_CR1,0,67,98_AL_.jpg",The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch transfers control of his clandestine empire to his reluctant son.,100,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
"https://m.media-amazon.com/images/M/MV

To use the Gemini API, you'll need an API key. If you don't already have one, create a key in Google AI Studio.
In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`. Then pass the key to the SDK:

In [ ]:
!pip install -q google-generativeai

In [ ]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

try:
  GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
  if not GOOGLE_API_KEY:
    raise ValueError("API key not found.")
  genai.configure(api_key=GOOGLE_API_KEY)
  print("Gemini API configured successfully.")
except ValueError as e:
  print(e)
  print("Please add your API key to Colab secrets named GOOGLE_API_KEY.")

Gemini API configured successfully.


Before you can make any API calls, you need to initialize a Generative Model.

In [ ]:
try:
  gemini_model = genai.GenerativeModel('gemini-1.5-flash-latest')
  print("Gemini model initialized.")
except Exception as e:
  print(f"Error initializing Gemini model: {e}")
  print("Please ensure your API key is valid and the model name is correct.")

Gemini model initialized.


# Task
Create an ADK interface for the movie recommendation system using Gradio. The interface should take a user query as input and display intelligent movie recommendations generated by the system, explaining why each movie is relevant. Use the existing functions for loading data, building the ChromaDB index, and querying for recommendations. The dataset is located at "/content/imdb_top_1000.csv".

## Define the adk tool

### Subtask:
Create a Python class that inherits from `google_adk.AgentTool` and define the tool's specification (name, description, parameters).


**Reasoning**:
I need to define a Python class that inherits from `google_adk.AgentTool` and specify its interface using `ToolSpecification` and `ToolParameter`. I will import the necessary classes and then define the `MovieRecommendationTool` class with its `tool_spec`.



**Reasoning**:
The previous command failed because the `google_adk` library is not installed. I need to install this library before I can import and use its classes.



In [ ]:
!pip install google-adk

**Reasoning**:
The `google-adk` library is confirmed to be installed. Now I can proceed with defining the `MovieRecommendationTool` class as specified in the instructions.



In [ ]:
!pip install --upgrade google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 47.9 MB/s  0:00:00
  Attempting uninstall: google-adk
    Found existing installation: google-adk 1.12.0
    Uninstalling google-adk-1.12.0:
      Successfully uninstalled google-adk-1.12.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [google-adk]


**Reasoning**:
Despite the previous installation attempt, the `google_adk` module is still not found. I will try installing it again, explicitly stating that I want to upgrade if it's already present, just in case there was an issue with the previous installation or environment.



**Reasoning**:
The `google-adk` library has now been successfully installed. I can proceed with defining the `MovieRecommendationTool` class as originally intended.



## Define the adk tool

### Subtask:
Create a Python class that inherits from `google_adk.AgentTool` and define the tool's specification (name, description, parameters).


**Reasoning**:
Since the previous attempt to install google-adk failed, I will try installing it again to ensure the necessary library is available for creating the AgentTool class.



In [ ]:
!pip install google-adk

**Reasoning**:
Since the previous installation attempt confirmed the library is already satisfied, I will proceed with defining the `MovieRecommendationTool` class as per the instructions, including the tool specification and a placeholder `call` method.



In [ ]:
from google_adk import AgentTool, ToolParameter, ToolSpecification

class MovieRecommendationTool(AgentTool):
    tool_spec = ToolSpecification(
        name="movie_recommendation_tool",
        description="Provides movie recommendations based on a user query.",
        parameters=[
            ToolParameter(
                name="query",
                type="string",
                description="The user's search query for movie recommendations.",
                required=True
            )
        ],
    )

    def call(self, query: str) -> str:
        """Placeholder call method - will be implemented later."""
        # This method will contain the logic to call the movie recommendation system
        # For now, it's a placeholder to satisfy the AgentTool requirement
        return f"Searching for movies based on: {query}"

ModuleNotFoundError: No module named 'google_adk'

**Reasoning**:
Despite the successful installation message, the `ModuleNotFoundError` persists. This indicates a deeper environment issue where the installed package is not accessible. I will try installing the package again, but this time I will also try to upgrade pip and setuptools first to ensure the installation process is using the latest versions.



In [ ]:
!pip install --upgrade pip setuptools
!pip install google-adk

## Implement the tool's logic

### Subtask:
Implement the `call` method within the `MovieRecommendationTool` class to utilize the existing movie recommendation functions and format the results.


**Reasoning**:
Implement the `call` method of the `MovieRecommendationTool` class to query movies, generate explanations using Gemini, and format the results.



In [ ]:


def call(self, query: str) -> str:
        """Provides movie recommendations based on a user query and explains the relevance."""
        try:
            # Query ChromaDB for similar movies
            hits = query_movies(query, collection, top_k=TOP_K)

            if not hits:
                return "No relevant movies found for your query."

            results_string = f"--- Recommended Movies for '{query}' ---\n"

            # Generate explanations and format results
            for i, h in enumerate(hits):
                title = h.get('title', 'Unknown Title')
                movie_idx = h.get('movie_idx')
                distance = h.get('distance', float('inf'))

                if movie_idx is not None and movie_idx in df.index:
                    full_description = df.loc[movie_idx, 'description']
                else:
                    full_description = h.get('matched_snippet', '') # Fallback

                explanation = "Could not generate explanation."
                try:
                    prompt = f"""
You are a witty movie guide recommending films.
Explain why the movie '{title}' with the description '{full_description}'
is an exciting match for the query '{query}'.

- Do NOT repeat the same sentence structure each time.
- Make the explanation engaging and a bit creative (like a movie critic or storyteller).
- Highlight what makes the movie unique or appealing to someone with that query.
- Keep the explanation short (2–3 sentences).
"""
                    explanation_response = gemini_model.generate_content(prompt)
                    explanation = explanation_response.text
                except Exception as e:
                    explanation = f"Could not generate explanation: {e}. Matched snippet: {h.get('matched_snippet', '')[:200]}..."


                results_string += f"\nResult {i+1}: title={title}\n"
                results_string += f"Explanation: {explanation}\n"
                results_string += f"Distance: {distance:.4f}\n"

            results_string += "\n--- End of Recommendations ---"
            return results_string

        except Exception as e:
            return f"An error occurred during the recommendation process: {e}"


**Reasoning**:
The `google_adk` module was not found. Re-install the library to make it available.



In [ ]:
!pip install -q google-generativeai

**Reasoning**:
Re-define the `MovieRecommendationTool` class with the implemented `call` method, as the previous attempt failed due to a missing module.



In [ ]:
def call(self, query: str) -> str:
        """Provides movie recommendations based on a user query and explains the relevance."""
        try:
            # Query ChromaDB for similar movies
            hits = query_movies(query, collection, top_k=TOP_K)

            if not hits:
                return f"🎬 Sorry, I couldn’t find any relevant movies for: '{query}'. Try another keyword!"

            results_string = f"🍿 --- Movie Recommendations for '{query}' --- 🍿\n"

            # Loop through recommendations
            for i, h in enumerate(hits):
                title = h.get('title', 'Unknown Title')
                movie_idx = h.get('movie_idx')
                distance = h.get('distance', float('inf'))

                # Get description from DataFrame if index is valid
                if movie_idx is not None and movie_idx in df.index:
                    full_description = df.loc[movie_idx, 'description']
                else:
                    full_description = h.get('matched_snippet', 'No description available.')

                # Generate creative explanation using Gemini
                try:
                    prompt = f"""
You are a witty movie guide recommending films. Be playful, engaging, and avoid monotony.
Explain why the movie '{title}' with the description '{full_description}'
is an exciting match for the query '{query}'.

Guidelines:
- Vary your sentence structure for each recommendation.
- Write like a movie critic or storyteller, with some flair.
- Highlight what makes this movie stand out.
- Keep it short: 2–3 sentences max.
- Start with something catchy, not "This movie is relevant because..."
"""
                    explanation_response = gemini_model.generate_content(prompt)
                    explanation = explanation_response.text.strip()
                except Exception as e:
                    explanation = f"(⚠️ Could not generate explanation: {e}. Matched snippet: {h.get('matched_snippet', '')[:200]}...)"

                # Format the recommendation block
                results_string += f"\n🎥 Recommendation {i+1}: **{title}**\n"
                results_string += f"✨ Why watch it: {explanation}\n"
                results_string += f"📏 Similarity score: {distance:.4f}\n"

            results_string += "\n--- End of Recommendations --- 🍿"
            return results_string

        except Exception as e:
            return f"🚨 An error occurred during the recommendation process: {e}"


## Wrap the tool

### Subtask:
Wrap the `MovieRecommendationTool` class using `google_adk.tool` to make it available as an ADK tool.


**Reasoning**:
Import the tool decorator from google_adk.



**Reasoning**:
The previous attempt to import `google_adk` failed. Try installing the library again to ensure it's available in the environment.



In [ ]:
!pip install google-adk

**Reasoning**:
Apply the `@tool` decorator directly above the `MovieRecommendationTool` class definition.



## Create a gradio interface

### Subtask:
Set up a Gradio interface that allows the user to input a search query and displays the output from the ADK tool.


**Reasoning**:
The previous attempts to use `google_adk` failed due to import errors. The current subtask is to set up a Gradio interface that calls the ADK tool. Since the ADK tool itself could not be successfully defined or imported, I cannot directly use it in the Gradio interface as intended by the instructions. To fulfill the spirit of the subtask (setting up a Gradio interface to display recommendations), I will create a function that directly calls the existing movie recommendation logic (`query_movies` and Gemini explanation) without relying on the problematic `google_adk` library. This function will then be used with Gradio.



In [ ]:
import gradio as gr

def get_movie_recommendations(query: str) -> str:
    """
    Gets movie recommendations based on a query and formats the results.
    This function bypasses the ADK tool wrapper due to import issues.
    """
    if not query:
        return "Please enter a movie query."

    try:
        # Query ChromaDB for similar movies using the existing function
        # 'collection' and 'df' are assumed to be globally available from previous steps
        hits = query_movies(query, collection, top_k=TOP_K)

        if not hits:
            return "No relevant movies found for your query."

        results_string = f"--- Recommended Movies for '{query}' ---\n"

        # Generate explanations and format results
        for i, h in enumerate(hits):
            title = h.get('title', 'Unknown Title')
            movie_idx = h.get('movie_idx')
            distance = h.get('distance', float('inf'))

            if movie_idx is not None and movie_idx in df.index:
                full_description = df.loc[movie_idx, 'description']
            else:
                 # Fallback if movie_idx is missing or invalid
                full_description = h.get('matched_snippet', '')


            explanation = "Could not generate explanation."
            try:
                # Use the existing gemini_model object
                prompt = f"""
You are a witty movie guide recommending films.
Explain why the movie '{title}' with the description '{full_description}'
is an exciting match for the query '{query}'.

- Do NOT repeat the same sentence structure each time.
- Make the explanation engaging and a bit creative (like a movie critic or storyteller).
- Highlight what makes the movie unique or appealing to someone with that query.
- Keep the explanation short (2–3 sentences).
"""
                explanation_response = gemini_model.generate_content(prompt)
                explanation = explanation_response.text
            except Exception as e:
                explanation = f"Could not generate explanation: {e}. Matched snippet: {h.get('matched_snippet', '')[:200]}..."

            results_string += f"\nResult {i+1}: title={title}\n"
            results_string += f"Explanation: {explanation}\n"
            results_string += f"Distance: {distance:.4f}\n"

        results_string += "\n--- End of Recommendations ---"
        return results_string

    except Exception as e:
        return f"An error occurred during the recommendation process: {e}"

# Create the Gradio interface
iface = gr.Interface(
    fn=get_movie_recommendations,
    inputs=gr.Textbox(label="Enter your movie query:"),
    outputs=gr.Textbox(label="Movie Recommendations:", lines=10),
    title="Movie Recommendation System"
)

# Launch the interface
iface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://00d2dbc024b3a5cd58.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Run the gradio app

### Subtask:
Launch the Gradio application to interact with the movie recommendation system.


**Reasoning**:
Launch the Gradio interface to allow user interaction with the movie recommendation system.



In [ ]:
iface.launch()

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://20d1f0d56ffb91d1f4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Summary:

### Data Analysis Key Findings

*   Attempts to import the `google_adk` library consistently resulted in a `ModuleNotFoundError`, despite `pip install` commands indicating successful installation or that requirements were already satisfied. This issue persisted across multiple steps.
*   Due to the persistent `ModuleNotFoundError` for `google_adk`, the original plan to define, implement, and wrap the movie recommendation logic using `google_adk.AgentTool` and `@tool` decorator could not be completed.
*   A workaround was successfully implemented by creating a standard Python function that directly integrated the existing movie recommendation logic (querying ChromaDB and generating explanations using the Gemini model).
*   A Gradio interface was successfully created and launched using this custom function, providing a functional web interface for users to input movie queries and receive recommendations.
*   The launched Gradio application is accessible via a public URL.

### Insights or Next Steps

*   Investigate the `ModuleNotFoundError` for `google_adk` in the execution environment to understand why the installed package is not being found. This might involve checking the Python path, virtual environment settings, or potential conflicts.
*   If the `google_adk` issue cannot be resolved in this environment, consider alternative methods for creating agent tools or focus on the current Gradio interface as a viable solution for user interaction.
